In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime as dt

%matplotlib inline

In [53]:
df1 = pd.read_csv("/Users/stephan/Data_Science/Metis/Bootcamp/Data_project1/turnstile_190504.txt")
df2 = pd.read_csv("/Users/stephan/Data_Science/Metis/Bootcamp/Data_project1/turnstile_190511.txt")
df3 = pd.read_csv("/Users/stephan/Data_Science/Metis/Bootcamp/Data_project1/turnstile_190518.txt")
df4 = pd.read_csv("/Users/stephan/Data_Science/Metis/Bootcamp/Data_project1/turnstile_190525.txt")

We found that the `EXITS` column contains trailing spaces, so we get rid of those after concatenating

In [54]:
raw_data = pd.concat([df1,df2,df3,df4])
raw_data.columns = raw_data.columns.str.rstrip()

The data is organized by turnstile, but does not have a direct unique identifier for each turnstile. To create one we add the column `ID`, which concatenates into a unique turnstile ID.

In [55]:
# Take the date and time fields into a single datetime column
raw_data["DATE_TIME"] = pd.to_datetime(raw_data.DATE + " " + raw_data.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

In [56]:
raw_data["ID"] = raw_data['C/A']+raw_data['UNIT']+raw_data['SCP']+raw_data['STATION']

Duplicates

In [57]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(raw_data
 .groupby(["ID", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,ID,DATE_TIME,ENTRIES
118612,G009R15102-00-04CONEY IS-STILLW,2019-05-16 17:00:00,2
118444,G009R15102-00-03CONEY IS-STILLW,2019-05-16 17:00:00,2
408244,N525R14201-00-03DELANCEY/ESSEX,2019-05-11 05:00:00,2
408415,N525R14201-00-04DELANCEY/ESSEX,2019-05-11 05:00:00,2
0,A002R05102-00-0059 ST,2019-04-27 00:00:00,1


In [58]:
# Get rid of the duplicate entry
raw_data.sort_values(["ID", "DATE_TIME"], 
                          inplace=True, ascending=False)
raw_data.drop_duplicates(subset=["ID", "DATE_TIME"], inplace=True)

In our design phase we decided to give our recommendations to the client in the form of stations to canvas, so we will group our data by station. We also move towards daily entries and exits instead of per 4 hours.

The data given for `ENTRIES` and `EXITS` is cumulative, so aggregating by the max for each day gives us the cumulative count at the end of each day.

In [72]:
data_by_station = raw_data.groupby(['STATION','ID','DATE'])['ENTRIES', 'EXITS'].first().diff()
data_by_station.reset_index()
data_by_station.dropna(subset=["ENTRIES","EXITS"], axis=0, inplace=True)

<ipython-input-72-e4c4f74c44e4>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_by_station = raw_data.groupby(['STATION','ID','DATE'])['ENTRIES', 'EXITS'].first().diff()


In [98]:
def get_daily_counts_entries(row, max_counter):
    if row["ENTRIES"] < 0:
        # Maybe counter is reversed?
        row["ENTRIES"] = abs(row["ENTRIES"])
    if row["ENTRIES"] > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        row["ENTRIES"] = 0
    return row["ENTRIES"]

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
# It'd probably be a good idea to use a number even significantly smaller than 1 million as the limit!
data_by_station["ENTRIES"] = data_by_station.apply(get_daily_counts_entries, axis=1, max_counter=86400)
# 1 person per second through individual turnstile, 86400 seconds in a day

In [96]:
test = data_by_station.groupby(['STATION','ID','DATE'])['ENTRIES'].fillna(method='ffill')

In [97]:
test.loc['RIT-ROOSEVELT']

ID                              DATE      
TRAM2R46900-00-00RIT-ROOSEVELT  04/27/2019       NaN
                                04/28/2019    1353.0
                                04/29/2019    1511.0
                                04/30/2019    1250.0
                                05/01/2019    1336.0
                                               ...  
TRAM2R46900-05-01RIT-ROOSEVELT  05/20/2019       0.0
                                05/21/2019       0.0
                                05/22/2019       0.0
                                05/23/2019       0.0
                                05/24/2019       0.0
Name: ENTRIES, Length: 168, dtype: float64